# __Gradient Descent with scaled MET features__

## __Import Packages and Tools__

import numpy as np

import pandas as pd

<br />

import metpy.calc as mpcalc

from metpy.units import units

<br />

from sklearn.preprocessing import MinMaxScaler,StandardScaler

from math import sqrt, exp

from sklearn.metrics import mean_squared_error

<br />

from sklearn.linear_model import SGDRegressor

from sklearn.model_selection import GridSearchCV

<br />

from yellowbrick.regressor import ResidualsPlot

---

## __Import Dataset(s)__

X_test = pd.read_csv('../data/file_test.csv')


X_train = pd.read_csv('../data/file_train.csv')

---

## __Overview Dataset(s)__

X_train.head()


X_test.head()

<br />

print('Train:', file_train.shape)

print('Test:', file_test.shape)

---

## __Define y_train und y_test__

---

## __Create Dummy Features if needed__

X_train = pd.get_dummies(X_train, drop_first=True)


X_test = pd.get_dummies(X_test, drop_first=True)


print('X Train:', X_train.shape)


print('X Test:', X_test.shape)

y_train = file_train['target']

y_test = file_test['target']

<br />

X_train = file_train.drop(['target', 'further_columns_if_needed'], axis = 1)

X_test = file_test.drop(['target', 'further_columns_if_needed'], axis = 1)

<br />

print('y Train:', y_train.shape)

print('y Test:', y_test.shape)

print('X Train:', X_train.shape)

print('X Test:', X_test.shape)

---

## __Scaling with Standard Scaler__

scaler = StandardScaler()

X_train_met_scaled = scaler.fit_transform(X_train_met)

X_test_met_scaled = scaler.transform(X_test_met)

---

## __Train the Model__

#### Train the unlog model

sgd = SGDRegressor(alpha=0.1, learning_rate='adaptive', penalty='elasticnet')

sgd.fit(X_train_met_scaled, y_train)

#### Train model log

sgd_log = SGDRegressor(alpha=0.1, learning_rate='adaptive', penalty='elasticnet')

sgd_log.fit(X_train_met_scaled, y_train_log)

---

## __Predictions__

#### Predict on test and train set
y_pred_train_sgd = sgd.predict(X_train_met_scaled)

y_pred_test_sgd = sgd.predict(X_test_met_scaled)

<br />

print('RMSE Train:', sqrt(mean_squared_error(y_train, y_pred_train_sgd)))

print('RMSE Test:', sqrt(mean_squared_error(y_test, y_pred_test_sgd)))

#### Predict on test and train set log
y_pred_train_sgd_log = sgd_log.predict(X_train_met_scaled)

y_pred_test_sgd_log = sgd_log.predict(X_test_met_scaled)

<br />

print('RMSE Train:', sqrt(mean_squared_error(y_train, y_pred_train_sgd)))

print('RMSE Test:', sqrt(mean_squared_error(y_test, y_pred_test_sgd)))

---
---

# __Error Analysis / Residuals Plots__

---

## __Residuals Plot with scaled, unlog data__

visualizer = ResidualsPlot(sgd)

#### Fit the training data to the visualizer
visualizer.fit(X_train_met_scaled, y_train)  

#### Evaluate the model on the test data
visualizer.score(X_test_met_scaled, y_test)  


visualizer.show()

---

## __Residuals Plot with scaled, log transformed data__

visualizer = ResidualsPlot(sgd_log)

#### Fit the training data to the visualizer
visualizer.fit(X_train_met_scaled, y_train_log)  

#### Evaluate the model on the test data
visualizer.score(X_test_met_scaled, y_test_log)  
visualizer.show()

---
---

# __GridSearch__

---

## __Prepare GridSearch__

#### Defining parameter grid (as dictionary)
param_grid = {'n_neighbors':[5,10],
              'weights':['uniform'],
              'p':[1,2,3],
              'leaf_size':[20,40]
             }

#### Instantiate gridsearch and define the metric to optimize 
gs_KNN_log = GridSearchCV(KNeighborsRegressor(), param_grid, scoring='neg_root_mean_squared_error',cv=5, verbose=2, n_jobs=-1)

#### Fit gridsearch object to data.. also lets see how long it takes
gs_KNN_log.fit(X_train_met_scaled_min_max, y_train_log)

#### Best Score and Parameters

print('Best score:', round(gs_KNN_log.best_score_, 3))

print('Best parameters:', gs_KNN_log.best_params_)

---

## __Predictions__

y_pred_train_gs_knn_log = gs_KNN_log.predict(X_train_met_scaled_min_max)

y_pred_test_gs_knn_log = gs_KNN_log.predict(X_test_met_scaled_min_max)

print('RMSE Train log:', sqrt(mean_squared_error(y_train_log, y_pred_train_gs_knn_log)))

print('RMSE Test log:', sqrt(mean_squared_error(y_test_log, y_pred_test_gs_knn_log)))